In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import keras
from tensorflow.keras.utils import to_categorical
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/FinalDatasetWithlabels.csv')#,index_col='time_s',parse_dates=['time_s'])

In [ ]:
# df = df.drop(['Unnamed: 0','subj_id'], 1)
df = df.drop(['Unnamed: 0'], 1)
df = df.drop(['magnitude'], 1)

In [ ]:
cleanup_nums = {"loc_id":   {"left_wrist":1,"left_hip":2,"left_ankle": 3, "right_ankle": 4}
                }
df = df.replace(cleanup_nums)                                  
df['loc_id'].unique()

array([1, 2, 3, 4])

In [ ]:
df = df.sort_values(['subj_id', 'loc_id'], ascending=[True, True])
df

,subj_id,loc_id,time_s,x,y,z,label
723888,id00b70b13,1,0.00,-0.344,-1.285,-0.039,0.0
723889,id00b70b13,1,0.01,-0.336,-1.301,-0.027,0.0
723890,id00b70b13,1,0.02,-0.305,-1.324,-0.016,0.0
723891,id00b70b13,1,0.03,-0.273,-1.371,-0.004,0.0
723892,id00b70b13,1,0.04,-0.227,-1.418,0.020,0.0
...,...,...,...,...,...,...,...
2590443,idff99de96,4,206.71,0.098,1.059,0.090,0.0
2590444,idff99de96,4,206.72,0.105,1.070,0.074,0.0
2590445,idff99de96,4,206.73,0.125,1.094,0.090,0.0
2590446,idff99de96,4,206.74,0.141,1.129,0.090,0.0


In [ ]:
df['subj_id'] = df['subj_id'].astype('category')
df['subj_id'] = df['subj_id'].cat.codes

df['label'] = df['label'].astype('category')
df['label'] = df['label'].cat.codes

In [ ]:
df

,subj_id,loc_id,time_s,x,y,z,label
723888,0,1,0.00,-0.344,-1.285,-0.039,0
723889,0,1,0.01,-0.336,-1.301,-0.027,0
723890,0,1,0.02,-0.305,-1.324,-0.016,0
723891,0,1,0.03,-0.273,-1.371,-0.004,0
723892,0,1,0.04,-0.227,-1.418,0.020,0
...,...,...,...,...,...,...,...
2590443,31,4,206.71,0.098,1.059,0.090,0
2590444,31,4,206.72,0.105,1.070,0.074,0
2590445,31,4,206.73,0.125,1.094,0.090,0
2590446,31,4,206.74,0.141,1.129,0.090,0


# ***SCALING***

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# define min max scaler
scaler = MinMaxScaler()

# df1 = df[['loc_id','magnitude','label']]
# df1

In [ ]:
df_scaled = pd.DataFrame(scaler.fit_transform(df),columns=df.columns)

# df = pd.concat([df['subj_id'], df1], axis=1)
df = df_scaled

In [ ]:
df

,subj_id,loc_id,time_s,x,y,z,label
0,0.0,0.0,0.000000,0.504167,0.371755,0.431236,0.0
1,0.0,0.0,0.000041,0.504779,0.370469,0.432446,0.0
2,0.0,0.0,0.000083,0.507149,0.368620,0.433555,0.0
3,0.0,0.0,0.000124,0.509596,0.364842,0.434765,0.0
4,0.0,0.0,0.000166,0.513113,0.361064,0.437185,0.0
...,...,...,...,...,...,...,...
2590443,1.0,1.0,0.855836,0.537962,0.560164,0.444243,0.0
2590444,1.0,1.0,0.855877,0.538497,0.561048,0.442630,0.0
2590445,1.0,1.0,0.855919,0.540026,0.562977,0.444243,0.0
2590446,1.0,1.0,0.855960,0.541249,0.565791,0.444243,0.0


# **TRAINING AND TEST DATA**



*   Total data   : 32 Subjects
*   Training data: 25 subjects
*   Test data    : 7 subjects



In [ ]:
df.subj_id.unique()[:25]

array([0.        , 0.03225806, 0.06451613, 0.09677419, 0.12903226,
       0.16129032, 0.19354839, 0.22580645, 0.25806452, 0.29032258,
       0.32258065, 0.35483871, 0.38709677, 0.41935484, 0.4516129 ,
       0.48387097, 0.51612903, 0.5483871 , 0.58064516, 0.61290323,
       0.64516129, 0.67741935, 0.70967742, 0.74193548, 0.77419355])

In [ ]:
df_train = df[df.subj_id.isin( df.subj_id.unique()[:25] ) ] 

In [ ]:
df_test = df[df.subj_id.isin( df.subj_id.unique()[25:] )]

# ***Every subject was timed for 15minutes ***

# **overlapping windows with 50% overlap - Window size :50 Step size:50**

In [ ]:
from scipy import stats
x_list = []
y_list = []

train_labels = []
new_list = []

window_size = 50
step_size = 50

for i in range(0, df_train.shape[0] - window_size, step_size):
    xs = df_train['x'].values[i: i + 50]
    xs1 = df_train['y'].values[i: i + 50]
    xs2 = df_train['z'].values[i: i + 50]
    # ys = df_train['loc_id'].values[i: i + 50]
    # zs = df_train['subj_id'].values[i: i + 50]
    label = stats.mode(df_train['label'][i: i + 50])[0][0]
    a = np.c_[ xs,xs1,xs2 ]
    new_list.append(a)
    train_labels.append(label)

Xtrain = np.array(new_list)

In [ ]:
train_x = np.asarray(Xtrain)
train_y = np.asarray(train_labels)

# ***CNN***

In [ ]:
model_cnn = Sequential() 

model_cnn.add(Conv1D(filters=160, kernel_size=2, activation='relu', input_shape=(50,3)))

# model_cnn.add(MaxPooling1D(pool_size=1))
model_cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
# model_cnn.add(MaxPooling1D(pool_size=1))
# model_cnn.add(Conv1D(filters=96, kernel_size=2, activation='relu'))
# model_cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=1))
model_cnn.add(Flatten())
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(64, activation='sigmoid'))
model_cnn.add(Dense(1,activation='sigmoid'))

model_cnn.compile(loss='binary_crossentropy', optimizer="adam", metrics = ["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 49, 160)           1120      
                                                                 
 conv1d_1 (Conv1D)           (None, 48, 128)           41088     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 48, 128)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 6144)              0         
                                                                 
 dropout (Dropout)           (None, 6144)              0         
                                                                 
 dense (Dense)               (None, 64)                393280    
                                                        

In [ ]:
model_cnn.fit(train_x, train_y, epochs=100, batch_size=1000, verbose=1)

Epoch 1/100
41/41 [==============================] - 12s 54ms/step - loss: 0.6972 - accuracy: 0.5024 - recall: 0.3694 - precision: 0.4930
Epoch 2/100
41/41 [==============================] - 2s 37ms/step - loss: 0.6933 - accuracy: 0.5068 - recall: 0.2397 - precision: 0.4984
Epoch 3/100
41/41 [==============================] - 1s 37ms/step - loss: 0.6935 - accuracy: 0.5054 - recall: 0.3449 - precision: 0.4968
Epoch 4/100
41/41 [==============================] - 1s 37ms/step - loss: 0.6933 - accuracy: 0.5039 - recall: 0.2744 - precision: 0.4932
Epoch 5/100
41/41 [==============================] - 1s 36ms/step - loss: 0.6932 - accuracy: 0.5061 - recall: 0.1994 - precision: 0.4962
Epoch 6/100
41/41 [==============================] - 1s 36ms/step - loss: 0.6932 - accuracy: 0.5065 - recall: 0.0780 - precision: 0.4932
Epoch 7/100
41/41 [==============================] - 1s 36ms/step - loss: 0.6931 - accuracy: 0.5021 - recall: 0.2713 - precision: 0.4899
Epoch 8/100
41/41 [=====================

## ***Evaluation***

Test data

In [ ]:
from scipy import stats
test_y = []
test_x = []

window_size = 50
step_size = 50

for i in range(0, df_test.shape[0] - window_size, step_size):
    xs = df_test['x'].values[i: i + 50]
    xs1 = df_test['y'].values[i: i + 50]
    xs2 = df_test['z'].values[i: i + 50]
    # zs = df_train['subj_id'].values[i: i + 50]
    # ys = df_test['loc_id'].values[i: i + 50]
    label = stats.mode(df_test['label'][i: i + 50])[0][0]
    a = np.c_[ xs,xs1,xs2 ]
    test_x.append(a)
    test_y.append(label)

In [ ]:
test_x = np.array(test_x)
test_y = np.array(test_y)

scores = model_cnn.evaluate(test_x,test_y, verbose=1)

353/353 [==============================] - 2s 6ms/step - loss: 0.6953 - accuracy: 0.5131 - recall: 0.1027 - precision: 0.5127


# ***Prediction***

In [ ]:
pred = model_cnn.predict(test_x) 
pred = np.argmax(pred, axis = 1)
label = test_y

x = pd.DataFrame(pred,columns= ["preds"])


In [ ]:
test_y

array([0., 1., 1., ..., 1., 0., 0.])

# ***Confusion Matrix***

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, pred)

array([[5761,    0],
       [5522,    0]])

## **Classification Report**

In [ ]:
from sklearn import metrics 
from sklearn.metrics import classification_report
print("\n -------------Classification Report-------------\n")
print(classification_report(test_y, pred))


 -------------Classification Report-------------

              precision    recall  f1-score   support

         0.0       0.51      1.00      0.68      5761
         1.0       0.00      0.00      0.00      5522

    accuracy                           0.51     11283
   macro avg       0.26      0.50      0.34     11283
weighted avg       0.26      0.51      0.35     11283



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
